**Import the required libraries**

In [ ]:
pip install pyarabic

In [ ]:
#-- not all the imported lib are used in this notebook --#
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from nltk.corpus import stopwords
import sys
import nltk
from nltk.tokenize import word_tokenize
#Import arabic libraries
import pyarabic.araby as araby
import pyarabic.number as number
import argparse

#Part1

1-Removing the dublicates

In [ ]:
#df=pd.read_csv('/content/in-class8.csv',usecols=['date','username','name','tweet'])

In [ ]:
df=pd.read_excel('/content/ALL.xlsx',usecols=['date','username','name','tweet','events'])

In [ ]:
#dublicates in tweet column
df.tweet.duplicated().sum()#we can remove the "sum" to see what columns that are duplicated

In [ ]:
df.duplicated().sum()#to see dublicates in general not only specific column 

In [ ]:
df.loc[df.duplicated(keep='last'), :]# to see what are these columns - 'first' can also be changed to last depends on what cloumns you want to consider as first appeared or use False 

In [ ]:
#df.duplicated(subset=['tweet','name']).sum()

In [ ]:
#df.tweet.drop_duplicates(keep='first').shape # 'first' 'last' False

In [ ]:
print(df[~df.duplicated(subset=['tweet'])]) # the ~ sign is to remove the dublicated fields! ,'name'

In [ ]:
df[~df.duplicated(subset=['tweet'])].to_csv('8_dropduplicated.csv',index=False)#save the cleaned data

2-Drop rows with certine value

In [ ]:
df=pd.read_excel('/content/ALL_p3.xlsx',usecols=['date','username','name','tweet'])

In [ ]:
#Drop the row if the "name"  contains any keyword mentioned in "searchfor"
searchfor = ['مدينة','دليل','غرفة','تأجير','بلدية','كلية','أنباء','موقع','منوعات','تعليم','مبادرة','وظيفة','مجمع','الأن','واتس','مشاهد','موجز','تحديث','وسم','الكرة','سناب','محافظة','إمارة','شركة','عكاظ','الاهرام','هاشتاق','اليوم','الجزائرية','المغربية','إجازة','اجازة','إدارة','رتويت','عناوين','إذاعة','channel','news','معهد','جامعة','مدرسة','حقائق','دليل','الوطن','اون لاين','محتوى','خطابة','اليمنية','السورية','الاردنية','الكويتية','الاماراتية','العربية','الدولية','الخليجية','شبكة','اونلاين','نيوز','صحيفة', 'جريدة' ,'قناة ','مجلة','جوال','القاهرة' ,'الالكترونية','مواعيد','حجوزات','حجز','أخبار','اخبار']
data = df[~df.name.str.contains('|'.join(searchfor), na=False)]

In [ ]:
len(data)

In [ ]:
data = df[~df.name.str.contains('|'.join(searchfor), na=False)].to_excel('drop1.xlsx')

In [ ]:
df=pd.read_excel('/content/drop1.xlsx',usecols=['date','username','name','tweet','events'])

In [ ]:
#Drop the row if the "name"  contains any keyword from list1 except it has any keyword from list2 in "tweet" then keep it.
List1 = ['_q8','🇶🇦','🇰🇼','Q8','uae','kuwait','leban','syri','qata','qtri','lib','yem','oman','Kuwaiti','egy','Egyp','egypt','q8','🇪🇬','rtarabic','🇧🇭','iraq','🇵🇸','online','🇦🇪']
List2 = ['ksa','Saudi','السعودية','توكلنا','Tawakkalna','المملكة']
data1=data[~data["name"].astype(str).str.contains('|'.join(List1)) | data["tweet"].astype(str).str.contains('|'.join(List2))]

In [ ]:
len(data1)

In [ ]:
#Drop the row if the "username"  contains any keyword from list1 except it has any keyword from list2 in "tweet" then keep it.
List1 = ['_q8','q8','Q8','kw','uae','kuwait','leban','syri','qata','qtri','lib','yem','oman','Kuwaiti','egy','egypt','rtarabic','bh','iraq','online']
List2 = ['ksa','Saudi','السعودية','توكلنا','Tawakkalna','المملكة']
data2= data1[~data1["username"].astype(str).str.contains('|'.join(List1)) | data1["tweet"].astype(str).str.contains('|'.join(List2))]

In [ ]:
len(data2)

In [ ]:
data2.to_excel('multipleq_clean_part3.xlsx')

#Part2

In [ ]:
df=pd.read_excel('/content/ALL.xlsx', usecols=['date','tweet'])#Reading the data file and show only tweet column
df.head()#show the file content
#,'username','name'

In [ ]:
#df.loc[df['tweet'].str.contains('@', case=False), 'tweet'] = '@username'

Remove Dicritics and normalization 

In [ ]:
def normalize_arabic(text):
    text = re.sub("[إأآ]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text;

In [ ]:
df["tweet"]=df["tweet"].apply(lambda x : normalize_arabic(x) )

In [ ]:
print(df)#no need but i like to see the result step by step!

In [ ]:


arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [ ]:
def remove_diacritics(text):
    text = re.sub(arabic_diacritics,"", text)
    return text;

In [ ]:
  df["tweet"]=df["tweet"].apply(lambda x : remove_diacritics(x) )

In [ ]:
df.head()

In [ ]:
# this code removes all words contains underscore from the dataset
def filter_string(str):
  a=[]
  for i in str.split():
        if "_" not in i:
            a.append(i)
  return ' '.join(a)

df["tweet"]=df["tweet"].apply(lambda x : filter_string(x) )

In [ ]:
df.head()



> **clean data from:**


* Numbers
*Upper and lower case english letters
*Newline 

In [ ]:
#arabic_numbers = '۰۱۲۳٤٥٦٧۸۹'
def clean_tweets(text):
 
 text=re.sub("[0-9]","",text)#Remove Numbers
 text=re.sub("[a-zA-Z]","",text)#all english letters
 text=re.sub("_"," ",text) #Already take care of in the previous step!
 text=re.sub("\n","",text)#remove newline
 #text=re.sub("[+arabic_numbers+]","",text)
 return text;

#arabic_numbers = '۰۱۲۳٤٥٦٧۸۹'
#df['tweet']= df['tweet'].str.replace("["+arabic_numbers+"]",'')
#df

In [ ]:
  df["tweet"]=df["tweet"].apply(lambda x : clean_tweets(x) )# save the result

In [ ]:
  print(df)# show the result

In [ ]:
df['tweet'] = df['tweet'].str.replace(r'\W'," ")#Remove Special "Char" after removing the diacritics to avoid deleteing the text contains diacritics
print(df)

In [ ]:
df['tweet'] = df['tweet'].str.replace(r' +'," ") #Remove more than one space betweeen the words
print(df)

In [ ]:
#df=pd.read_csv('/content/processed_new_vaccine.csv')
df.to_excel("/content/ALL_clean.xlsx")# save in excel format

**Data Preprocessing**

In [ ]:
df=pd.read_excel('/content/all datacollected.xlsx', usecols=['sentiment','tweet'])
df.head()

In [ ]:
df['sentiment'] = df.sentiment.map({'neutral':2, 'negative':0, 'positive':1})
print(df.shape)# to get the size
df.head() 